# Volt Analytics

## Import Dependencies

In [1]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import openpyxl as Workbook

pd.set_option('display.max_rows', 500)

In [2]:
currentDate =  dt.datetime.today().date()
currentDate

datetime.date(2017, 8, 22)

## Read in CSV files

In [3]:
#Volt Daily Data
voltDailyData = pd.read_csv('data/voltdailydata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltDailyData['Date'] = pd.to_datetime(voltDailyData['Date'])
voltDailyData['Date'] = voltDailyData['Date'].dt.date

In [4]:
voltDailyData.head()

,Date,MilesDriven,EvMilesDriven,GallonsBurned
0,2017-06-25,163.7,52.6,3.03005
1,2017-06-26,202.4,42.5,4.15543
2,2017-06-27,202.3,53.8,3.97314
3,2017-06-28,197.7,52.4,3.93617
4,2017-06-29,192.6,45.2,3.83577


In [5]:
# Calculate total ICE (Internal Combustion Engine) miles driven
voltDailyData = voltDailyData.rename(columns = {'MilesDriven':'TotalMilesDriven'})
voltDailyData['TotalICEMiles'] = voltDailyData['TotalMilesDriven'] - voltDailyData['EvMilesDriven']
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles
0,2017-06-25,163.7,52.6,3.03005,111.1
1,2017-06-26,202.4,42.5,4.15543,159.9
2,2017-06-27,202.3,53.8,3.97314,148.5
3,2017-06-28,197.7,52.4,3.93617,145.3
4,2017-06-29,192.6,45.2,3.83577,147.4


In [6]:
# Create new DailyMPG Column, round values after adding new column
voltDailyData['DailyMPG'] = voltDailyData['TotalMilesDriven'] / voltDailyData['GallonsBurned']
voltDailyData['DailyMPG'] = pd.to_numeric(voltDailyData['DailyMPG'], errors='coerse')
voltDailyData = voltDailyData.replace(np.nan, 0)
voltDailyData = voltDailyData.round(2)
voltDailyData.head()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
0,2017-06-25,163.7,52.6,3.03,111.1,54.030000
1,2017-06-26,202.4,42.5,4.16,159.9,48.710000
2,2017-06-27,202.3,53.8,3.97,148.5,50.920000
3,2017-06-28,197.7,52.4,3.94,145.3,50.230000
4,2017-06-29,192.6,45.2,3.84,147.4,50.210000
5,2017-06-30,12.2,0.0,0.35,12.2,35.250000
6,2017-07-01,119.5,33.3,2.80,86.2,42.640000
7,2017-07-02,97.9,25.5,1.87,72.4,52.340000
8,2017-07-03,35.4,9.0,0.75,26.4,47.350000
9,2017-07-04,10.8,10.8,0.00,0.0,inf


In [7]:
# Add '250+' miles to any DailyMPG value that has a value of infinity
# Why 250+? That's the "infinity mileage" that the volt displays in car

voltDailyData = voltDailyData.replace(np.inf, '250+')
voltDailyData.tail()

,Date,TotalMilesDriven,EvMilesDriven,GallonsBurned,TotalICEMiles,DailyMPG
47,2017-08-17,121.5,59.2,1.81,62.3,67.04
48,2017-08-18,0.0,0.0,0.00,0.0,0
49,2017-08-19,0.0,0.0,0.00,0.0,0
50,2017-08-20,7.4,2.5,0.13,4.9,56.03
51,2017-08-21,189.6,53.0,3.76,136.6,50.37


In [8]:
voltDailyData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 6 columns):
Date                52 non-null object
TotalMilesDriven    52 non-null float64
EvMilesDriven       52 non-null float64
GallonsBurned       52 non-null float64
TotalICEMiles       52 non-null float64
DailyMPG            52 non-null object
dtypes: float64(4), object(2)
memory usage: 2.5+ KB


## Volt Readings

In [9]:
#Volt Readings throughout the day (usually two), convert Timestamp to datetime object
voltReadings = pd.read_csv('data/voltdata_' + currentDate.strftime('%Y%m%d') + '.csv')
voltReadings.rename(columns = {'LocalTimestamp':'Date', 'GallonsBurned': 'LifeTimeGalBurned'}, inplace=True)
voltReadings['Date'] = pd.to_datetime(voltReadings['Date'])

voltReadings.head()

,Date,BatteryLevel,ChargeState,OilLife,TirePressure_RF,TirePressure_RR,TirePressure_LF,TirePressure_LR,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeEvMiles,LifetimeMiles,LastTripFuelEcon,LastTripEvMiles,LastTripMiles,GallonsOfGas
0,2017-08-22 18:39:50,0.0,NaN,0.0,0.0,0.0,0.0,0.0,726.86926,65.45950,20097.95288,47580.49847,0.0,0.0,0.0,7.11415
1,2017-08-22 08:51:41,0.0,NaN,0.0,0.0,0.0,0.0,0.0,724.76381,65.51192,20076.50315,47480.66897,0.0,0.0,0.0,2.91382
2,2017-08-21 19:04:35,0.0,NaN,0.0,0.0,0.0,0.0,0.0,722.86969,65.54683,20041.04771,47381.81503,0.0,0.0,0.0,4.85284
3,2017-08-21 05:59:27,0.0,NaN,0.0,0.0,0.0,0.0,0.0,719.10524,65.62626,19988.04475,47192.19118,0.0,0.0,0.0,8.57766
4,2017-08-20 17:18:56,0.0,NaN,0.0,0.0,0.0,0.0,0.0,718.97316,65.62810,19985.50334,47184.84036,0.0,0.0,0.0,3.42631


In [10]:
# Select only columns desired
voltReadings = voltReadings[['Date', 'LifeTimeGalBurned', 'LifetimeFuelEcon', 'LifetimeMiles', 'GallonsOfGas']]

In [13]:
# Sort based on timestamp, round values
voltReadings.sort_values(by='LifetimeMiles', inplace=True)
voltReadings = voltReadings.round(2)
voltReadings.tail()

,Date,LifeTimeGalBurned,LifetimeFuelEcon,LifetimeMiles,GallonsOfGas
4,2017-08-20 17:18:56,718.97,65.63,47184.84,3.43
3,2017-08-21 05:59:27,719.11,65.63,47192.19,8.58
2,2017-08-21 19:04:35,722.87,65.55,47381.82,4.85
1,2017-08-22 08:51:41,724.76,65.51,47480.67,2.91
0,2017-08-22 18:39:50,726.87,65.46,47580.50,7.11


In [12]:
voltReadings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 117 entries, 116 to 0
Data columns (total 5 columns):
Date                 117 non-null datetime64[ns]
LifeTimeGalBurned    117 non-null float64
LifetimeFuelEcon     117 non-null float64
LifetimeMiles        117 non-null float64
GallonsOfGas         117 non-null float64
dtypes: datetime64[ns](1), float64(4)
memory usage: 5.5 KB
